In [1]:
import pandas as pd
import numpy as np
from KDEpy import FFTKDE
import matplotlib.pyplot as plt
from scipy.optimize import minimize
from scipy.optimize import Bounds
from statsmodels.nonparametric.bandwidths import bw_silverman
import math
from KDE import TKDE
from sklearn.neighbors import KernelDensity
import datetime

In [2]:
dax    = pd.read_csv(r'data/returns_dax.csv')
nq     = pd.read_csv(r'data/returns_nq.csv')
dow    = pd.read_csv(r'data/returns_dow.csv')
crix   = pd.read_csv(r'data/returns_crix.csv')
snp500 = pd.read_csv(r'data/returns_snp500.csv')
crix   = crix.rename(columns={'date': 'Date'})
btc    = pd.read_csv(r'data/returns_btc.csv')

returns = pd.DataFrame({'timestamp' : dax.Date,
                       'dax'       : dax['returns'],
                       'dow'       : dow['returns'],
                       'crix'      : crix.returns[1:].reset_index(drop=True),
                       'snp500'    : snp500['returns'],
                       'nq'        : nq['returns'],
                       'btc'       : btc['returns'][1:-1].reset_index(drop=True)})

returns.index = pd.to_datetime(returns['timestamp'])
returns.drop('timestamp', axis=1, inplace = True)
returns.loc['2019-01-02'] = 0

### Before COVID-19

In [3]:
dic_list = returns.apply(lambda index: TKDE(index, until='2020-01-01'))

`gtol` termination condition is satisfied.
Number of iterations: 12, function evaluations: 16, CG iterations: 7, optimality: 5.60e-10, constraint violation: 0.00e+00, execution time: 0.08 s.
`gtol` termination condition is satisfied.
Number of iterations: 12, function evaluations: 16, CG iterations: 7, optimality: 1.68e-09, constraint violation: 0.00e+00, execution time: 0.074 s.
`gtol` termination condition is satisfied.
Number of iterations: 11, function evaluations: 14, CG iterations: 6, optimality: 6.97e-09, constraint violation: 0.00e+00, execution time: 0.063 s.
`gtol` termination condition is satisfied.
Number of iterations: 12, function evaluations: 16, CG iterations: 7, optimality: 1.03e-09, constraint violation: 0.00e+00, execution time: 0.066 s.
`gtol` termination condition is satisfied.
Number of iterations: 12, function evaluations: 16, CG iterations: 7, optimality: 3.95e-10, constraint violation: 0.00e+00, execution time: 0.063 s.
`gtol` termination condition is satisfied

### After COVID-19

In [4]:
dic_list_covid = returns.apply(lambda index: TKDE(index, start='2020-01-01', until='2020-12-31'))

`gtol` termination condition is satisfied.
Number of iterations: 12, function evaluations: 16, CG iterations: 7, optimality: 6.02e-10, constraint violation: 0.00e+00, execution time: 0.07 s.
`gtol` termination condition is satisfied.
Number of iterations: 12, function evaluations: 16, CG iterations: 7, optimality: 2.08e-10, constraint violation: 0.00e+00, execution time: 0.067 s.
`gtol` termination condition is satisfied.
Number of iterations: 12, function evaluations: 16, CG iterations: 7, optimality: 2.38e-09, constraint violation: 0.00e+00, execution time: 0.067 s.
`gtol` termination condition is satisfied.
Number of iterations: 12, function evaluations: 16, CG iterations: 7, optimality: 2.95e-10, constraint violation: 0.00e+00, execution time: 0.064 s.
`gtol` termination condition is satisfied.
Number of iterations: 12, function evaluations: 16, CG iterations: 7, optimality: 2.11e-10, constraint violation: 0.00e+00, execution time: 0.064 s.
`gtol` termination condition is satisfied

In [5]:
df_pre_COVID   = pd.DataFrame({'x'           : np.linspace(-38, 19, 2000),
                             'dax_TVKDE'     : dic_list['dax']['TVKDE'], 
                             'nq_TVKDE'      : dic_list['nq']['TVKDE'],
                             'dow_TVKDE'     : dic_list['dow']['TVKDE'],
                             'crix_TVKDE'    : dic_list['crix']['TVKDE'],
                             'snp500_TVKDE'  : dic_list['snp500']['TVKDE'],
                             'btc_TVKDE'     : dic_list['btc']['TVKDE'],
                             'dax_U'         : dic_list['dax']['Uweights'], 
                             'nq_U'          : dic_list['nq']['Uweights'],
                             'dow_U'         : dic_list['dow']['Uweights'],
                             'crix_U'        : dic_list['crix']['Uweights'],
                             'snp500_U'      : dic_list['snp500']['Uweights'],
                             'btc_U'         : dic_list['btc']['Uweights']})

In [6]:
df_post_COVID = pd.DataFrame({'x'            : np.linspace(-38, 19, 2000),
                             'dax_TVKDE'     : dic_list_covid['dax']['TVKDE'], 
                             'nq_TVKDE'      : dic_list_covid['nq']['TVKDE'],
                             'dow_TVKDE'     : dic_list_covid['dow']['TVKDE'],
                             'crix_TVKDE'    : dic_list_covid['crix']['TVKDE'],
                             'snp500_TVKDE'  : dic_list_covid['snp500']['TVKDE'],
                             'btc_TVKDE'     : dic_list_covid['btc']['TVKDE'],
                             'dax_U'         : dic_list_covid['dax']['Uweights'], 
                             'nq_U'          : dic_list_covid['nq']['Uweights'],
                             'dow_U'         : dic_list_covid['dow']['Uweights'],
                             'crix_U'        : dic_list_covid['crix']['Uweights'],
                             'snp500_U'      : dic_list_covid['snp500']['Uweights'],
                             'btc_U'         : dic_list_covid['btc']['Uweights']})

In [7]:
df_post_COVID.to_csv('data/densities_post_covid.csv', index=False)
df_pre_COVID.to_csv('data/densities_pre_covid.csv', index=False)